## import all necessary libraries

In [187]:
import pandas as pd
import numpy as np
import seaborn as sns

## Data ingestion

In [188]:
#reading the data
data = pd.read_csv('Data/train.csv')
data.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [189]:
#dropping the id column
data =  data.drop(labels=['id'],axis=1)

In [190]:
data.shape

(193573, 10)

##Dividing the dataset in to dependent and independent features
X = data.drop(labels=['price'],axis=1)

y = data[['price']]


In [191]:
#dividng data in to dependent and independent features
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

In [192]:
X.shape

(193573, 9)

In [193]:
y.shape

(193573,)

In [194]:
y

0         13619
1         13387
2          2772
3           666
4         14453
          ...  
193568     1130
193569     2874
193570     3036
193571      681
193572     2258
Name: price, Length: 193573, dtype: int64

In [195]:
#segregating numerical and categorical_features
categorical_features = X.select_dtypes(include='object').columns
numerical_features = X.select_dtypes(exclude='object').columns

In [196]:
# Define the custom ranking for each ordinal variable
cut_categories = ['Fair', 'Good', 'Very Good','Premium','Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [197]:
from sklearn.impute import SimpleImputer ## HAndling Missing Values
from sklearn.preprocessing import StandardScaler #for feature scaling
from sklearn.preprocessing import OrdinalEncoder # for encoding categrical values

In [198]:
#for pipeline[means combining two setps]
from sklearn.pipeline import Pipeline # for pipelining
from sklearn.compose import ColumnTransformer

## column transformer
ColumnTransformer enables us to apply transform to particular columns. It help us to fit multiple transformations to multiple columns with a single fit() or fit_transform() statement. For example, we can impute mean in column 1 and OneHotEncode column 2 with a single fit statement.

## Numerical pipeline

In [199]:
numerical_pipeline = Pipeline(steps = [('imputer',SimpleImputer(strategy='median')),('scaler',StandardScaler())])

## categorical pipeline

In [200]:
categorical_piepline = Pipeline(
    steps = [
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinal_encoding',OrdinalEncoder(categories=[cut_categories,color_categories,clarity_categories])),
        ('scaler',StandardScaler())
        ]
        )

## combining numerical and categorical pipelines

In [201]:
pp = ColumnTransformer([
    ('num_pipeline',numerical_pipeline,numerical_features),
    ('cat_pipeline',categorical_piepline,categorical_features)
])

## Train test split

In [202]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.30,random_state=35)

In [203]:
pp.fit_transform(X_train)

array([[ 0.60557075,  0.25929759, -0.6410321 , ...,  0.8746223 ,
        -0.93347189, -1.31412765],
       [ 0.45411871,  0.25929759,  1.96561571, ..., -3.15168006,
         0.29890036, -1.9792092 ],
       [-0.75749767, -0.57497871, -0.6410321 , ...,  0.8746223 ,
         0.29890036,  2.01128013],
       ...,
       [-0.1949615 , -2.15083396,  0.40162702, ..., -0.13195329,
         0.29890036,  0.68111702],
       [ 0.60557075,  0.25929759, -0.11970254, ...,  0.8746223 ,
         0.91508649,  0.01603546],
       [ 1.64409908,  0.44469233,  0.40162702, ..., -1.13852888,
         1.53127261, -0.64904609]])

In [205]:
#converting the above array to dataframe
X_train = pd.DataFrame(pp.fit_transform(X_train),columns = pp.get_feature_names_out())

In [206]:
X_train.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,0.605571,0.259298,-0.641032,0.754024,0.727308,0.777613,0.874622,-0.933472,-1.314128
1,0.454119,0.259298,1.965616,0.609707,0.491299,0.589180,-3.151680,0.298900,-1.979209
2,-0.757498,-0.574979,-0.641032,-0.734242,-0.715977,-0.758845,0.874622,0.298900,2.011280
3,-0.476230,-0.018795,-0.641032,-0.328352,-0.361964,-0.323998,0.874622,0.298900,1.346199
4,0.237759,-3.077808,1.965616,0.609707,0.518531,0.270292,-2.145104,-0.317286,-0.649046


In [207]:
X_test = pd.DataFrame(pp.transform(X_test),columns = pp.get_feature_names_out())

In [208]:
X_test.head()

,num_pipeline__carat,num_pipeline__depth,num_pipeline__table,num_pipeline__x,num_pipeline__y,num_pipeline__z,cat_pipeline__cut,cat_pipeline__color,cat_pipeline__clarity
0,-0.822406,0.073903,-1.162362,-0.815421,-0.870290,-0.831319,0.874622,-1.549658,0.681117
1,-1.060402,-0.853071,1.444286,-1.257391,-1.242458,-1.295155,-0.131953,0.915086,1.346199
2,0.237759,0.444692,-0.641032,0.357153,0.427758,0.429736,0.874622,-0.933472,-0.649046
3,-0.844042,1.186271,2.486945,-0.959737,-0.942908,-0.860309,-2.145104,0.298900,-0.649046
4,-0.800770,0.351995,-0.641032,-0.851500,-0.833981,-0.816824,0.874622,-0.933472,0.016035


## Model training

In [210]:
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

In [212]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [ ]:
regression.predict()

In [213]:
regression.coef_

array([ 6410.61764913,  -129.16761137,   -65.2954592 , -2025.62869326,
        -137.41618944,  -100.19804034,    73.57303664,  -463.71986462,
         652.94239617])

In [214]:
regression.intercept_

3963.4344838783472

In [225]:
#metrics of model 
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse,r2_square

In [232]:
models = {"linear_regression":LinearRegression(),"ridge":Ridge(),"lasso":Lasso(),"elaticnet":ElasticNet()}
#creating key -value pair for each model
model_list = []
r2score = []
for i in range(len(models)):
    model = list(models.values())[i]#getting values in dictionary which are intiliazed objects of model
    model.fit(X_train,y_train)#fitting the model
    y_pred = model.predict(X_test)#making the prediction 
    mae,rmse,r2_square = evaluate_model(y_test,y_pred)#calling the above function which for metrics
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)
    r2score.append(r2_square)
    print('='*35)
    print('\n')




linear_regression
Model Training Performance
RMSE: 1018.3046961936127
MAE: 678.1484356451721
R2 score 93.67943932734687


ridge
Model Training Performance
RMSE: 1018.2948621210945
MAE: 678.1750542671697
R2 score 93.67956140584087


lasso
Model Training Performance
RMSE: 1018.5953874733459
MAE: 679.3578926270582
R2 score 93.67583020303418


elaticnet
Model Training Performance
RMSE: 1546.3666068107339
MAE: 1068.7548170724851
R2 score 85.42444714354535


